In [1]:
import os
import sys
import datetime
import re
import time

import numpy as np
import pandas as pd

import warnings
# 遍历所有文件
warnings.filterwarnings('ignore') 


"""
参数配置
"""

# 保存路径
save_path = r'C:\Users\vkad\Desktop\result\mx_vs_lp.xlsx'

# 明细表目录
mx_dir = r"C:\Users\vkad\Desktop\签约明细表"

# 财务楼盘目录
cwlp_dir = r"C:\Users\vkad\Desktop\财务楼盘"


# 明细表参与运算查字段
mx_fileds = ['合并房号','工作簿名','姓名','类型','签约日期','key']


#财务楼盘表参与运算字段
# cwlp_fields = ['房间ID', '项目名称', '分期名称', '产品类型全称', '营销楼栋名称','房号', '房间编码','房间工程名称','相关房源','客户名称']
cwlp_fields = [ '项目名称','房间编码', '分期名称', '产品类型全称', '营销楼栋名称','房号','客户名称','相关房源','合同编号','网签日期']

MXVSLP_FIELDS = ['项目简称','房间编码','楼栋','房号','客户名称','相关房源','lp_key']

MAP_MXVSLP_FIELDS = ['项目简称','房间编码','楼栋','房号','姓名','相关房源','lp_key']


# 楼盘参数

# 楼盘表项目检查提取方案
lp_xm_patt = '(?P<项目简称>梅溪郡|滨江|白鹭郡|国际|紫台|缇香)'
# 提取 需要分析的项目
lp_xm_sx = [ '长沙金域滨江', '长沙梅溪郡', '长沙白鹭郡', '长沙紫台', '长沙金域国际','长沙金域缇香']


"""
读取最新的楼盘报表
"""

format_date = '%Y-%m-%d %H_%M_%S'
com_patt = '^财务楼盘表(?P<date>20\d{2}-\d{2}-\d{2} \d{2}_\d{2}_\d{2})\.xlsx$'
def update_path(dir_name,com_patt,format_date,extend_name = 'xlsx'):
    patt = com_patt
    com = re.compile(patt)
    date = pd.to_datetime('1900-1-1')
    file_name = None
    for p,d,fs_name in os.walk(cwlp_dir):
        for f_name in fs_name:
            x = com.match(f_name)
            if x:
                str_d = x.groupdict().get('date')
                temp_date = pd.to_datetime(str_d,format=format_date)
                date = date 
                if date < temp_date:
                    file_name = f_name
                    date = temp_date
                
    if file_name:

        return os.path.join(cwlp_dir,file_name)
    else:
        return None


path = update_path(cwlp_dir,com_patt,format_date)
if path:
    print('读取最新楼盘报表')
    start_time = time.time()
    df_lp = pd.read_excel(path,sheet_name='财务楼盘表0')
    end_time = time.time()

    print('耗时%0.2f秒' %(end_time - start_time))

else:
    print('请确认财务楼盘目录')
    sys.exit()





读取最新楼盘报表
耗时114.01秒


In [2]:
"""
分析商业和住宅
"""


fields = cwlp_fields
df_t = df_lp[fields]


# 提取楼栋
df_t['楼栋'] = df_t['营销楼栋名称'].str.extract('(?P<ld>[BS]?\d+)',expand=False)
# 获取项目简称
df_t['项目简称'] = df_t['项目名称'].str.extract(lp_xm_patt,expand=False).str.replace('国际','百汇')
df_t['类型_lp'] = df_t['产品类型全称'].str[:2]
df_t['lp_key']= df_t['项目简称'] + '-'+df_t['类型_lp'] + '-' + df_t['楼栋'] + '-'+ df_t['房号']
df_t['lp_key'] = df_t['lp_key'].str.replace('(?<=-)-','S')




df_t1 = df_t[df_t['项目名称'].isin(lp_xm_sx)]
df_t1 = df_t1[df_t1['产品类型全称'].str.contains('^住宅|^商业')]


#  合并表格
path = mx_dir
df_mx = pd.DataFrame()
patt = "(?!<$)(?P<area>.+)签约明细.+\.xls[xm]?"
com = re.compile(patt)

file_dic = {}
for i,j,k in os.walk(path):
    for x in k:
        z = com.match(x)
        if '~' not in x and '河西未转'not in x and z: 
            area = z.group('area')
            try:
                print('读取' + area + '商业签约明细')
                
                dfsy = pd.read_excel(path + "\/" + x,sheetname = area+"商业签约明细")
               
                dfsy['工作簿名'] = area

               
                dfsy.rename(columns={  '单机改网签约时间':'单机改网签日期'},inplace=True)
                dfsy['类型'] = '商业'
                print('合并' + area + '商业签约明细')
                df_mx = pd.concat([df_mx, dfsy]) 
            except Exception as e:
                print('->无'+ area + '商业签约明细表')
            
            try:
                print('读取' + area + '签约明细')
                dfzf = pd.read_excel(path + "\/" + x,sheetname = area+"签约明细")

                
                dfzf['工作簿名'] = area
                dfzf['类型'] = '住宅'
                dfzf.rename(columns={  '单机改网签约时间':'单机改网签日期'},inplace=True)
                print('合并' + area + '签约明细')
                df_mx = pd.concat([df_mx, dfzf])
            except Exception as e:
                print('--->无'+ area + '签约明细表')
           
            print()
    print('end')
            
df_mx.reset_index(drop=True,inplace=True)



"""
明细表字段处理

"""
# 分离字段
s = df_mx['项目名称']
dd = s.str.extract(r"\s*(?P<项目名称_trim>(?P<项目>.+?)(?P<项目期数>[一二三四五六七八九]期(?:商业)?))\s*")
df_mx1 = pd.concat([dd, df_mx],axis=1)

# #  检查 项目名称 这一列时候彻底分裂
# df_mx1[df_mx1['项目名称_trim']!= df_mx1['项目名称']][['工作簿名','项目名称_trim','项目','项目期数','项目名称']]

#  去除无效字段
df_mx1.dropna(subset=['项目名称'],inplace=True)
# 改回名字
del df_mx1['项目名称']
df_mx1.rename(columns={'项目名称_trim':'项目名称'},inplace=True)


# df_hb = df_mx1.copy
df_mx1['key'] = df_mx1['工作簿名']+'-'+df_mx1['类型']+'-'+df_mx1['合并房号']


"""
处理函数
"""

def not_unique_part(data,key):
    x = data[key].value_counts()
    return data[data[key].isin(x[x>1].index)].sort_values(key).reset_index(drop=True)

def unique_part(data,key):
    x = data[key].value_counts()
    return data[data[key].isin(x[x==1].index)].sort_values(key).reset_index(drop=True)

# 提取分析部分
check_fileds = mx_fileds
df_hb = df_mx1[check_fileds]

"""处理

"""


# 提取出 非唯一部分
mx_not_unique = not_unique_part(df_hb,'key')
cwlp_not_unique = not_unique_part(df_t1,'lp_key')

# 筛选出姓名不同的

df_t1_u = unique_part(df_t1,'lp_key')
df_com = pd.merge(df_hb,df_t1_u,how='left',left_on='key',right_on='lp_key')
df_com['姓名'] = df_com['姓名'].str.replace('[；;，]',',').str.strip()
df_com_t1 = df_com[df_com['姓名'] != df_com['客户名称']].reset_index(drop=True)

# 筛选出 两个名字 写反的部分
x = df_com_t1['姓名'].str.split(',')
df_com_t1['姓名'] = x.apply(lambda x:','.join([i.strip() for i in x[::-1]]))
df_com_t1=df_com_t1[df_com_t1['姓名'] != df_com_t1['客户名称']].reset_index(drop=True)

# 姓名需要 细比的部分
df_com_t2 = df_com_t1[df_com_t1['房号'].notnull()]


# 明细表 在 财务楼盘表找不到的部分
df_com_t3 = df_com_t1[df_com_t1['房号'].isnull()].reset_index(drop=True).sort_values('key')








读取梅溪郡商业签约明细
合并梅溪郡商业签约明细
读取梅溪郡签约明细
合并梅溪郡签约明细

读取滨江商业签约明细
->无滨江商业签约明细表
读取滨江签约明细
合并滨江签约明细

读取白鹭郡商业签约明细
合并白鹭郡商业签约明细
读取白鹭郡签约明细
合并白鹭郡签约明细

读取百汇商业签约明细
合并百汇商业签约明细
读取百汇签约明细
合并百汇签约明细

读取紫台商业签约明细
->无紫台商业签约明细表
读取紫台签约明细
合并紫台签约明细

读取缇香商业签约明细
合并缇香商业签约明细
读取缇香签约明细
合并缇香签约明细

end


In [3]:

# 读取车位
df_mx = pd.DataFrame()
patt = "(?!<$)(?P<area>.+)签约明细\d.+\.xlsx?"
com = re.compile(patt)

file_dic = {}
for i,j,k in os.walk(path):
    for x in k:
        z = com.match(x)
        if '~' not in x and z: 
            area = z.group('area')
            try:
                print('读取' + area + '车位明细')
                
                dfcw = pd.read_excel(path + "\/" + x,sheetname = area+"车位明细")
                
                dfcw['工作簿名'] = area

              
                dfcw['类型'] = '车位'
                print('合并' + area + '车位明细')
                df_mx = pd.concat([df_mx, dfcw]) 
            except Exception as e:
                print('->无'+ area + '车位明细')
           
            print()
    print('end')
            
df_mx.reset_index(drop=True,inplace=True)





读取梅溪郡车位明细
合并梅溪郡车位明细

读取滨江车位明细
合并滨江车位明细

读取白鹭郡车位明细
合并白鹭郡车位明细

读取百汇车位明细
合并百汇车位明细

读取紫台车位明细
合并紫台车位明细

读取缇香车位明细
合并缇香车位明细

end


In [4]:
df_cw = df_t[df_t['类型_lp']=="车位"].reset_index(drop=True)

# In[] 筛选出客户名称

df_kh = df_cw['客户名称'].value_counts().to_frame().\
sort_values('客户名称',ascending=False)

# user_l = df_kh.index[3:]
# df_cw = df_cw[df_cw['客户名称'].isin(user_l.to_list())]

df_cw['lp_key'] = df_cw['房号'] + "-" + df_cw['客户名称']

# In[]
mx_fileds = ['付款方式','车位号码','车位业主姓名','明源转签约日期','合并房号','地址' ,'工作簿名','车位签约日期']
 
df_mxcw = df_mx[mx_fileds]

df_mxcw['key'] = df_mxcw['车位号码'].astype(str) + "-" + df_mxcw['车位业主姓名'] 
df_mxcw['车位号码'] = df_mxcw['车位号码'].astype(str).str.replace("\s","").\
                    str.extract("([a-zA-Z0-9]+(?:[-=][a-zA-Z0-9]+)?)")
                    
df_mxcw['车位业主姓名'] = df_mxcw['车位业主姓名'].str.strip().str.replace("\s+|，|;|；",",")
df_mxcw['mx_key'] = df_mxcw['车位号码'].astype(str) + "-" + df_mxcw['车位业主姓名'] 

df_mxcw['mx_key'] = df_mxcw['mx_key'].str.strip("；;,，")
df_mxcw['mx_key'] = df_mxcw['mx_key'].str.replace("[a-zA-Z0-9]+=","")
df_mxcw['mx_key'] = df_mxcw['mx_key'].str.replace("([a-zA-Z])-",r"\1")
df_mx_not = not_unique_part(df_mxcw,'mx_key')
df_mxcw_unique= unique_part(df_mxcw,'mx_key')

# In[] 连接

df_hb = pd.merge(df_mxcw_unique,df_cw,how='left',left_on='mx_key',right_on='lp_key')
df_hb_null = df_hb[df_hb['房号'].isnull()]
df_hb_not_null = df_hb[df_hb['房号'].notnull()]
# In[]
df_hb_qianyue_null = df_hb_null[df_hb_null['车位签约日期'].isnull()]



In [5]:

"""保存"""

writer = pd.ExcelWriter(save_path)
df_t1_u_save = df_t1_u[MXVSLP_FIELDS]
df_t1_u_save.columns = MAP_MXVSLP_FIELDS

MXVSLP_FIELDS.append('key')
MAP_MXVSLP_FIELDS.append('cwlp_key')
df_hb_not_null_save = df_hb_not_null[MXVSLP_FIELDS]
df_hb_not_null_save.rename(columns = {"lp_key":"cwlp_key"},inplace=True)
df_hb_not_null_save.rename(columns = {"key":"lp_key"},inplace=True)
MXVSLP_FIELDS.pop()
MXVSLP_FIELDS.append('cwlp_key')

df_hb_not_null_save = df_hb_not_null_save[MXVSLP_FIELDS]


df_hb_not_null_save.columns = MAP_MXVSLP_FIELDS



pd.concat([df_t1_u_save,df_hb_not_null_save])[MAP_MXVSLP_FIELDS].to_excel(writer,sheet_name='roomcodemap',index=False)
if len(mx_not_unique):
    mx_not_unique.to_excel(writer,sheet_name='签约明细表中重复key',index=False) 
if len(cwlp_not_unique):
    cwlp_not_unique.to_excel(writer,sheet_name='财务楼盘重复key',index=False)
df_com_t2.to_excel(writer,sheet_name='姓名需细细比较',index=False)
df_com_t3.to_excel(writer,sheet_name='未在楼盘报表中',index=False)

df_mx_not.to_excel(writer,sheet_name="cw_明细重复部分",index=False)
df_hb_null.to_excel(writer,sheet_name="cw_不在楼盘表中",index=False)
df_hb_qianyue_null.to_excel(writer,sheet_name="cw_不在楼盘表签约为空",index=False)

writer.save()